# Neural Grammar Dependency Parsing
In this notebook, a neural transition-based dependency parser is implemented, based off the paper <a href="https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf">A Fast and Accurate Dependency Parser using Neural Networks</a>.

The setup for a dependency parser is somewhat more sophisticated than tasks like classification or translation.

## Load Packages

In [ ]:
import numpy as np
import math
import random
import time
from tqdm.notebook import tqdm
import cloudpickle as cp
from urllib.request import urlopen

from spacy import displacy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

## Prepare Data




### Read & Visualize Data

Preprocessed data is imported below. Here are some reference links about the data:
* A Universal Dependencies dataset: [UD_English dataset in version 1.4](http://universaldependencies.org/docsv1/)
* Refer to https://universaldependencies.org/ for more info about the Universal Dependencies framework in general.
* The data license can be found here: https://lindat.mff.cuni.cz/repository/xmlui/page/licence-UD-1.4

In [ ]:
def load_data():
    testing_bundle = cp.load(urlopen("https://github.com/wjonasreger/data/raw/main/grammar_parse/testing_bundle.pkl"))
    train_set = cp.load(urlopen("https://github.com/wjonasreger/data/raw/main/grammar_parse/train_set.pkl"))
    test_set = cp.load(urlopen("https://github.com/wjonasreger/data/raw/main/grammar_parse/dev_set.pkl"))
    return train_set, test_set

In [ ]:
train_set, test_set = load_data()
print("Num. Train Examples:", len(train_set))
print("Num. Test Examples: ", len(test_set))

Num. Train Examples: 12543
Num. Test Examples:  2002


Next, the training data is visualized, which contains labeled dependency trees. At test time, the goal is to predict the dependency arcs & labels for a given sentence.

In [ ]:
def display_sentence(sent):
    res = {'words': [{'text': "<ROOT>", 'tag': 'POS_ROOT'}], 'arcs': []}
    for i in range (len(sent['word'])):
        res['words'].append({'text': sent['word'][i], 'tag': sent['pos'][i]})
        s = i + 1
        e = sent['head'][i]
        direc = "left"
        if s > e: 
            s = sent['head'][i]
            e = i + 1
            direc = 'right'
        cur = {'start': s, 'end': e, 'label': sent['label'][i], 'dir': direc}
        res['arcs'].append(cur)
    displacy.render(res, style="dep", manual=True, jupyter=True, options={'distance': 70})


In [ ]:
MIN_SENT_LEN, MAX_SENT_LEN = 4, 17 # adjustable parameters
for x in random.sample(list(filter(lambda x: len(x['word']) >= MIN_SENT_LEN and len(x['word']) <= MAX_SENT_LEN, train_set)), 5):
    display_sentence(x)

### Build Vocabulary

Next, the `Vocabulary` class is built. This maps each word, part-of-speech tag (POS), and label to an id (index), which is later used in the embeddings. Each possible transition needs to be enumerated, and mapped to an id, since this is what the neural network will try to predict.

Suppose there are $n$ labels. For each label, a Left-Arc (LA) and Right-Arc (RA) action is created; and a separate Shift (S) action is created. This creates a total of $2n+1$ actions that the dependency parser will be able to choose from.

In [ ]:
class Vocabulary(object):
    def __init__(self, dataset):

        UNK = '<UNK>'
        NULL = '<NULL>'
        ROOT = '<ROOT>'

        # find the label of the root
        root_labels = list([l for ex in dataset for (h, l) in zip(ex['head'], ex['label']) if h == 0])
        assert len(set(root_labels)) == 1
        self.root_label = root_labels[0]

        # create mapping from transitions to ids
        labels = sorted(list(set([w for ex in dataset for w in ex['label'] if w != self.root_label]))) # list of unique non-root labels
        labels = [self.root_label] + labels # add root label too
        self.n_labels = len(labels)

        transitions = ['LA-' + l for l in labels] + ['RA-' + l for l in labels] + ['S']
        self.n_trans = len(transitions) # 2*n_labels + 1
        self.tran2id = {t: i for (i, t) in enumerate(transitions)}
        self.id2tran = {i: t for (i, t) in enumerate(transitions)}

        # create mapping from word, pos, & label to id
        # do labels first
        self.LABEL_PREFIX = '<l>:'
        self.tok2id = {self.LABEL_PREFIX + l: i for (i, l) in enumerate(labels)}
        self.LABEL_NULL = self.tok2id[self.LABEL_PREFIX + NULL] = len(self.tok2id) # add null label in

        # do pos's
        self.POS_PREFIX = '<p>:'
        all_pos = sorted(set([self.POS_PREFIX + w for ex in dataset for w in ex['pos']])) # get pos's
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_pos)}) # add poses in
        self.POS_NULL = self.tok2id[self.POS_PREFIX + NULL] = len(self.tok2id) # add null pos
        self.POS_ROOT = self.tok2id[self.POS_PREFIX + ROOT] = len(self.tok2id) # add root pos
        self.n_pos = 2 + len(all_pos) # +2 for null, root

        # do words
        all_word = sorted(set([w for ex in dataset for w in ex['word']]))
        self.tok2id.update({w: index + len(self.tok2id) for (index, w) in enumerate(all_word)}) # add words in
        self.WORD_UNK = self.tok2id[UNK] = len(self.tok2id) # add unk word
        self.WORD_NULL = self.tok2id[NULL] = len(self.tok2id) # add null word
        self.WORD_ROOT = self.tok2id[ROOT] = len(self.tok2id) # add root word
        self.n_words = 3 + len(all_word) # +3 for unk, null, root

        self.id2tok = {v: k for (k, v) in self.tok2id.items()} # flip it
        self.n_tokens = len(self.tok2id)

    def printStats(self):
        print('Num. labels:', self.n_labels)
        print('Num. transitions (2*n_labels + 1):', self.n_trans)
        print('Num. pos:', self.n_pos)
        print('Num. words:', self.n_words)
        print('Num. tokens:', self.n_tokens)


    def buildSentences(self, examples):
        processed_sentences = []
        for ex in examples:
            # initialize words & dependencies
            words = [Word('<ROOT>', '<POS_ROOT>', 0, self.WORD_ROOT, self.POS_ROOT)]
            deps = [] 
          
            # loop over words in sentence
            for i  in  range(len(ex['word'])):
                w = ex['word'][i]
                word_id = (self.tok2id[w] if w in self.tok2id else self.WORD_UNK)
                pos = self.POS_PREFIX + ex['pos'][i]
                pos_id = self.tok2id[pos]
                word = Word(ex['word'][i], ex['pos'][i],i+1, word_id, pos_id)
                
                words.append(word)
                deps.append((ex['head'][i], word, ex['label'][i] ))
            
            # create dependencies
            dependencies = Dependencies(self)
            for dep in deps:
                dependencies.add(words[dep[0]], dep[1], dep[2])

            processed_sentences.append((words, dependencies))
                      
        return processed_sentences

Here are some stats from the vocabulary.

In [ ]:
Vocabulary(train_set).printStats()

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735


## Parser Data Structures
In this section, some useful data structures for dependency parsing are defined. In particular, the `Stack` and `Buffer` structures are implemented, which make up a `ParserConfiguration`.

### Word, Arc & Dependencies

First, some useful data classes are defined.

In [ ]:
class Word(object):
    def __init__(self, word, pos, idx, word_id=None, pos_id=None):
        self.word = word
        self.pos = pos
        self.idx = idx
        self.word_id = word_id
        self.pos_id = pos_id

    def __str__(self):
        return 'Word(idx=' + str(self.idx) + ", word='" + self.word + "', pos='" + self.pos + \
        "', word_id=" + str(self.word_id) + ', pos_id=' + str(self.pos_id) + ')'

    def copy(self):
        return Word(self.word, self.pos, self.idx, self.word_id, self.pos_id)

    def __eq__(self, obj):
        if not isinstance(obj, Word): return False
        if obj.idx == self.idx:
            assert obj.word == self.word and obj.pos == self.pos and obj.word_id == self.word_id and obj.pos_id == self.pos_id, \
            'Your Word object has been corrupted.'
        return obj.idx == self.idx


class Arc(object):
    def __init__(self, head, dependent, label, label_id):
        self.head=head # word object
        self.dependent=dependent # word object
        self.label=label
        self.label_id = label_id

    def __str__(self):
        return 'Arc(head_idx=' + str(self.head.idx) + ', dep_idx=' + str(self.dependent.idx) + ', label_id=' + str(self.label_id) + ')'


class Dependencies(object):
    def __init__(self, vocab):
        self.arcs = []
        self.vocab = vocab
        self.dep_to_head_mapping = {} # for fast lookup
    
    def add(self, head, dependent, label):
        assert label[:3] != 'LA-' and label[:3] != 'RA-', 'Need to pass in just the label to add(...), not the entire action.'
        assert head is not None and dependent is not None, "Must pass in two Word objects to add(...)."
        self.arcs.append(Arc(head, dependent, label, self.vocab.tok2id[self.vocab.LABEL_PREFIX+label]))
        assert dependent.idx not in self.dep_to_head_mapping
        self.dep_to_head_mapping[dependent.idx] = self.arcs[-1]

    def getArcToHead(self, dependent):
        if dependent.idx == 0: # special case for ROOT
            return Arc(None, dependent, None, None)
        return self.dep_to_head_mapping[dependent.idx]

    def __iter__(self):
        return iter(self.arcs) # allows iteration "for x in Dependencies(...)"

### Stack & Buffer

Here, the stack and buffer data structures are defined.

In [ ]:
class Stack(object):
    def __init__(self, input=[]):
        self.stack = [word.copy() for word in input] 

    def push(self, item):
        self.stack.append(item.copy())

    def pop(self):
        assert len(self.stack) > 0
        return self.stack.pop()

    def get_si(self, i):
        assert i > 0, 'Must provide i > 0'
        return self.stack[-i] if len(self.stack) >= i else None

    def __getitem__(self, idx):
        return self.stack[idx]

    def __len__(self):
        return len(self.stack)

    def __str__(self):
        return str([str(x) for x in self.stack])


class Buffer(object):
    def __init__(self, sentence):
        self.buffer = [word.copy() for word in sentence]

    def pop(self):
        assert len(self.buffer) > 0
        return self.buffer.pop(0)

    def get_bi(self, i):
        assert i > 0, 'Must provide i > 0'
        return self.buffer[i-1] if len(self.buffer) >= i else None

    def __getitem__(self, idx):
        return self.buffer[idx]

    def __len__(self):
        return len(self.buffer)

    def __str__(self):
        return str([str(x) for x in self.buffer])

### Parser Configuration

Next, the `ParserConfiguration` class contains the `Stack`, `Buffer`, and `Dependencies` data structures.

Let $\sigma$ represent the stack, $\beta$ the buffer, and $A$ the set of arcs (dependencies). Based on the value of `transition`, `parse_step(self, transition)` does the following:
* `transition = 'S'`: &nbsp;<b>Shift</b> $w_k$ from the buffer to the stack. $(\sigma, w_k|\beta, A) \Rightarrow (\sigma|w_k, \beta, A)$
* `transition = 'LA-label'`: &nbsp;Add a <b>left arc</b> with label $label$ from $w_j$ to $w_i$. $(\sigma |w_i w_j , \beta, A) \Rightarrow (\sigma |w_j, \beta, A \cup \{(w_j, label, w_i)\})$
* `transition = 'RA-label'`: &nbsp;Add a <b>right arc</b> with label $label$ from $w_i$ to $w_j$. $(\sigma |w_i w_j , \beta, A) \Rightarrow (\sigma |w_i, \beta, A \cup \{(w_i, label, w_j)\})$

In [ ]:
class ParserConfiguration(object):
    def __init__(self, sentence, vocab):
        self.vocab = vocab

        assert sentence[0].word_id == self.vocab.WORD_ROOT
        self.stack = Stack([sentence[0]]) # initialize stack with ROOT
        self.buffer = Buffer(sentence[1:]) # initialize buffer with sentence
        self.dependencies = Dependencies(vocab)

    def parse_step(self, transition):
        assert transition in self.vocab.tran2id
        
        if transition == 'S': # shift
          self.stack.push(self.buffer.pop())
        elif transition[:3] == 'LA-': # left add arc
          keep_word = self.stack.pop()
          pop_word = self.stack.pop()
          self.stack.push(keep_word)
          self.dependencies.add(head=keep_word, dependent=pop_word, label=transition[3:])
        else: # right add arc
          pop_word = self.stack.pop()
          keep_word = self.stack.pop()
          self.stack.push(keep_word)
          self.dependencies.add(head=keep_word, dependent=pop_word, label=transition[3:])

## Generate Training Data

The dataset contains many sentences along with their gold dependency parses. In order to use a transition-based parsing algorithm, the next parser action at each time step is predicted, based on the current parser configuration. Thus, each sentence is transformed into a series of training examples, where the input features are based on the current parser configuration and the correct label is the gold action.

In this section, an algorithm to select the next parser action at each time step based on the gold dependency parse is implemented. Then, features from each step's parser configuration are extracted, which the neural network will use to predict the next action.

### Compute Gold Action

Next, the function `get_gold_action(stack, buffer, gold_dependencies)` is defined. Given a stack and buffer, this function returns the next action of the parser based on the gold dependencies.

Let $s_i$ be $i$th element of the stack and $h(s_i)$ be the head word of $s_i$.

1. If the stack only contains `ROOT`:
 - If the buffer is not empty, return `S` (shift).
 - If the buffer is empty, return `DONE`, indicating the parse is complete.
2. If $h(s_2)=s_1$, return `LA-label`.
3. If $h(s_1)=s_2$ <b>and</b> $h(b_i) \neq s_1$ for all words $b_i$ in the buffer, return `RA-label`.
4. Otherwise:
 - If the buffer is not empty, return `S`. 
 - If the buffer is empty, return `None`, indicating a failed parse (i.e., the sentence is non-projective).

In [ ]:
def get_gold_action(stack, buffer, gold_dependencies):
    action = None

    if len(stack) == 1:
      if len(buffer) > 0:
        action = 'S'
      else:
        action = 'DONE'
    else:
      s1, s2 = stack.get_si(1), stack.get_si(2)
      s1_arc, s2_arc = gold_dependencies.getArcToHead(s1), gold_dependencies.getArcToHead(s2)
      bi_heads = [gold_dependencies.getArcToHead(bi).head for bi in buffer]
      if s2_arc.head == s1:
        action = '-'.join(['LA', s2_arc.label])
      elif (s1_arc.head == s2) & (s1 not in bi_heads):
        action = '-'.join(['RA', s1_arc.label])
      elif len(buffer) > 0:
        action = 'S'
      else:
        action = action # None

    return action

### Generate Training Examples


Now a function to generate the training examples is defined. Recall that each sentence needs to be converted into a series of separate training examples. Each training example will essentially be a partial parser configuration along with its gold action; the goal of the neural network will be to predict this action from the parser configuration.

In [ ]:
def generate_training_examples(sentence, gold_dependencies, vocab, feat_extract = lambda parser_config: []):
    training_examples = []
    parser_config = ParserConfiguration(sentence, vocab)
    continue_parse = True
    
    while continue_parse:
      gold_action = get_gold_action(parser_config.stack, parser_config.buffer, gold_dependencies)
      if gold_action == 'DONE':
        continue_parse = False
      elif gold_action == None:
        return []
      else:
        training_examples.append((feat_extract(parser_config), gold_action))
        parser_config.parse_step(gold_action)

    return training_examples

The following function calls `generate_training_examples(...)` on every sentence in the dataset to create the full training data.

In [ ]:
def generate_all_training_examples(vocab, sentences, feat_extract = lambda parser_config: []):
    all_training_examples = []
    successful_sents = 0
    for sentence in sentences:
        training_examples = generate_training_examples(sentence[0], sentence[1], vocab, feat_extract)
        if training_examples != []:
            all_training_examples += training_examples
            successful_sents += 1

    print("Successfully generated training examples for", successful_sents, "/", len(sentences), "sentences")
    print("Number of training examples:", len(all_training_examples))

    return all_training_examples

In [ ]:
_vocab = Vocabulary(train_set) # Variable just used in this cell
train_examples = generate_all_training_examples(_vocab, _vocab.buildSentences(train_set))

Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


### Extract Features
So far, individual training instances can be created. Each instance is made up of a parser configuration along with the gold action that the classifier should be trained to predict.

In order to make this prediction, the neural network relies on features extracted from each parser configuration. This follows the procedure described at the end of Section 3.1 of <a href="https://nlp.stanford.edu/pubs/emnlp2014-depparser.pdf">A Fast and Accurate Dependency Parser using Neural Networks</a>. 

In total, 48 features are extracted from the parser configuration $-$ 18 word features, 18 POS features, and 12 label features:
* Word & POS features for $s_1$, $s_2$, $s_3$
* Word & POS features for $b_1$, $b_2$, $b_3$
* Word, POS, & label features for $lc_1(s_1)$, $lc_2(s_1)$, $lc_1(s_2)$, $lc_2(s_2)$
* Word, POS, & label features for $rc_1(s_1)$, $rc_2(s_1)$, $rc_1(s_2)$, $rc_2(s_2)$
* Word, POS, & label features for $lc_1(lc_1(s_1))$, $lc_1(lc_1(s_2))$, $rc_1(rc_1(s_1))$, $rc_1(rc_1(s_2))$

A separate function for each of the five extractions above are defined. Each function will return a list of word features, a list of POS features, and a list of label features.

A "feature" refers to the id (index in the Vocabulary) of a word, POS, or label. The neural network will then construct embeddings for each id.

First, two functions are defined to extract features corresponding to the words at the top of the stack & buffer:
* `get_top3_stack_features(parser_config)`: Returns word & POS features (ids) for $s_1$, $s_2$, $s_3$.
* `get_top3_buffer_features(parser_config)`: Returns word & POS features (ids) for $b_1$, $b_2$, $b_3$.

Wherever a particular word does not exist (such as when the stack or buffer has length $< 3$) the NULL token is used. This is necessary to ensure that the neural network will see an equally sized feature vector for each example.

In [ ]:
def get_top3_stack_features(parser_config):
    word_features, pos_features = [parser_config.vocab.WORD_NULL]*3, [parser_config.vocab.POS_NULL]*3
    si_word_ids = [si.word_id for si in parser_config.stack[-3:]]
    si_pos_ids = [si.pos_id for si in parser_config.stack[-3:]]
    
    for i in range(3):
      try:
        word_features[len(si_word_ids)-1-i] = si_word_ids[i]
        pos_features[len(si_pos_ids)-1-i] = si_pos_ids[i]
      except:
        pass

    return word_features, pos_features

In [ ]:
def get_top3_buffer_features(parser_config):
    word_features, pos_features = [parser_config.vocab.WORD_NULL]*3, [parser_config.vocab.POS_NULL]*3

    for i in range(3):
      try:
        word_features[i] = parser_config.buffer[i].word_id
        pos_features[i] = parser_config.buffer[i].pos_id
      except:
        pass
    
    return word_features, pos_features

The remaining features have to do with the leftmost & rightmost children of the words at the top of the stack & buffer. The following two helper functions are defined to make it easier to access these dependents:
* `get_lc(parser_config, word)`: Returns a list of arcs to dependents of `word`, sorted from <b>left to right</b>. Dependents that are to the <b>left</b> of `word` in the sentence are only included.
* `get_rc(parser_config, word)`: Returns a list of arcs to dependents of `word`, sorted from <b>right to left</b>. Dependents that are to the <b>right</b> of `word` in the sentence are only included.

In [ ]:
def get_lc(parser_config, word):
    word_deps = [dep for dep in parser_config.dependencies if (dep.head.idx == word.idx) and (dep.head.idx > dep.dependent.idx)]
    lc = sorted(word_deps, key=lambda arc: arc.dependent.idx)

    return lc

In [ ]:
def get_rc(parser_config, word):
    word_deps = [dep for dep in parser_config.dependencies if (dep.head.idx == word.idx) and (dep.head.idx < dep.dependent.idx)]
    rc = sorted(word_deps, key=lambda arc: arc.dependent.idx, reverse=True)

    return rc

Let $lc_j(s_i)$ be the $j$th leftmost child of the $i$th item on the stack. The following function is defined:
* `get_lc1_lc2_features(parser_config, i)`: Returns word & POS features for $lc_1(s_i)$ and $lc_2(s_i)$. Additionally, returns label features (the label ids) for the arcs that attach $lc_1(s_i)$ to $s_i$ and $lc_2(s_i)$ to $s_i$. As before, wherever a particular word does not exist, the NULL token is used.

In [ ]:
def get_lc1_lc2_features(parser_config, i):
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    try:
      si = parser_config.stack[-i]
      si_lc = get_lc(parser_config, si)
      for i in range(2):
        try:
          word_features[i] = si_lc[i].dependent.word_id
          pos_features[i] = si_lc[i].dependent.pos_id
          label_features[i] = si_lc[i].label_id
        except:
          pass
    except:
      pass
    
    return word_features, pos_features, label_features

The analagous function for the rightmost children is implemented below. Let $rc_j(s_i)$ be the $j$th rightmost child of the $i$th item on the stack.
* `get_rc1_rc2_features(parser_config, i)`: Returns word & POS features for $rc_1(s_i)$ and $rc_2(s_i)$. Additionally, returns label features (the label ids) for the arcs that attach $rc_1(s_i)$ to $s_i$ and $rc_2(s_i)$ to $s_i$. As before, wherever a particular word does not exist, the NULL token is used.

In [ ]:
def get_rc1_rc2_features(parser_config, i):
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    try:
      si = parser_config.stack[-i]
      si_rc = get_rc(parser_config, si)
      for i in range(2):
        try:
          word_features[i] = si_rc[i].dependent.word_id
          pos_features[i] = si_rc[i].dependent.pos_id
          label_features[i] = si_rc[i].label_id
        except:
          pass
    except:
      pass
    
    return word_features, pos_features, label_features

Finally, the last function is implemented to bring it aqll together:
* `get_llc_rrc_features(parser_config, i)`: Returns word & POS features for $lc_1(lc_1(s_i))$ and $rc_1(rc_1(s_i))$. Additionally, returns label features (the label ids) for the arcs that attach $lc_1(lc_1(s_i))$ to $lc_1(s_i)$ and $rc_1(rc_1(s_i))$ to $rc_1(s_i)$. As before, wherever a particular word does not exist, the NULL token is used.

In [ ]:
def get_llc_rrc_features(parser_config, i):
    assert i in {1,2}
    word_features, pos_features, label_features = [parser_config.vocab.WORD_NULL]*2, [parser_config.vocab.POS_NULL]*2, [parser_config.vocab.LABEL_NULL]*2

    try:
      si = parser_config.stack[-i]
      si_lc = get_lc(parser_config, si)
      lc1_lc = get_lc(parser_config, si_lc[0].dependent)
      try:
        word_features[0] = lc1_lc[0].dependent.word_id
        pos_features[0] = lc1_lc[0].dependent.pos_id
        label_features[0] = lc1_lc[0].label_id
      except:
        pass
    except:
      pass

    try:
      si = parser_config.stack[-i]
      si_rc = get_rc(parser_config, si)
      rc1_rc = get_rc(parser_config, si_rc[0].dependent)
      try:
        word_features[1] = rc1_rc[0].dependent.word_id
        pos_features[1] = rc1_rc[0].dependent.pos_id
        label_features[1] = rc1_rc[0].label_id
      except:
        pass
    except:
      pass
    
    return word_features, pos_features, label_features

The function `extract_features(parser_config)` is defined to call each of these functions and returns a list of the 48 total features.

In [ ]:
def extract_features(parser_config): # for both train & inference
    word_features, pos_features, label_features = [], [], []

    # 1. Get word & pos features for s1, s2, and s3
    (x, y) = get_top3_stack_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y

    # 2. Get word & pos features for b1, b2, and b3
    (x, y) = get_top3_buffer_features(parser_config)
    word_features, pos_features = word_features + x, pos_features + y

    # 3. Get word & pos & label features for lc1(s1), lc1(s2), lc2(s1), lc2(s2)
    (x, y, z) = get_lc1_lc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z
    (x, y, z) = get_lc1_lc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    # 4. Get word & pos & label features for rc1(s1), rc1(s2), rc2(s1), rc2(s2)
    (x, y, z) = get_rc1_rc2_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z
    (x, y, z) = get_rc1_rc2_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    # 5. Get word & pos & label features for lc1(lc1(s1)), lc1(lc1(s2)), rc1(rc1(s1)), rc1(rc1(s2))
    (x, y, z) = get_llc_rrc_features(parser_config, 1)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z
    (x, y, z) = get_llc_rrc_features(parser_config, 2)
    word_features, pos_features, label_features = word_features + x, pos_features + y, label_features + z

    features = word_features + pos_features + label_features
    assert len(features) == 48
    return features

## Dataset & Model

Now the Pytorch `TrainDataset` class and model are defined.

### Instantiate Dataset

A Pytorch `TrainDataset` class is defined, which is used to feed the training data to the model.

In [ ]:
class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, data, vocab):
        self.X = np.array([d[0] for d in data])
        self.y = np.array([vocab.tran2id[d[1]] for d in data])
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

### Define Model

Here a feed-forward network is defined with an embedding layer, a single hidden layer, and an output layer. The `forward(...)` method takes in the features extracted from the parser configuration and predict the next parser action.

In [ ]:
class Model(nn.Module):
    def __init__(self, num_embeddings, embed_size, n_features, hidden_size, n_classes, dropout_prob):
        '''
        Initialize the weights of feed-forward neural network.
        Args:
            num_embeddings: Number of embedding vectors in embedding layer (int)
            embed_size: Size of the embedding vectors in embedding layer (int)
            n_features: Number of features in the input to the model (int)
            hidden_size: Hidden size (int)
            n_classes: Number of classes in output (int)
            dropout_prob: Probability of dropout (float)
        '''
        super(Model, self).__init__()

        # Initialize embedding layer 
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embed_size)
        # Initialize a linear layer that maps the (concatenated) embeddings to a single vector of size hidden_size
        self.linmap = nn.Linear(n_features*embed_size, hidden_size)
        # Create a dropout layer with dropout_prob
        self.dropout = nn.Dropout(dropout_prob)
        # Initialize a linear layer that maps the hidden vector to the number of output classes
        self.out = nn.Linear(hidden_size, n_classes)

    def forward(self, x):
        '''
        This function predicts the next parser action, given the features extracted from the current parser state.
        Inputs:
             x: input features, [batch_size, n_features]
        Returns:
            logits: [batch_size, n_classes]
        '''

        x = self.embedding(x)
        x = self.linmap( torch.flatten(x, start_dim=1) )
        x = F.relu(x)
        x = self.dropout(x)
        output = self.out(x)

        return output

## Train Model

Finally, the model is ready to be trained.

First, the training dataset is read in, training examples are created, features are extracted, and the Pytorch `TrainDataset` is instantiated.

In [ ]:
def prepare_data(train_name='train', test_name='test'):

    train_set, test_set = load_data()

    vocab = Vocabulary(train_set)
    vocab.printStats()
    print()

    train_set = vocab.buildSentences(train_set)
    test_set = vocab.buildSentences(test_set)

    train_examples = generate_all_training_examples(vocab, train_set, feat_extract=extract_features)

    return vocab, train_examples, test_set

In [ ]:
vocab, train_examples, test_data = prepare_data()
train_dataset = TrainDataset(train_examples, vocab)

Num. labels: 47
Num. transitions (2*n_labels + 1): 95
Num. pos: 52
Num. words: 16635
Num. tokens: 16735

Successfully generated training examples for 11888 / 12543 sentences
Number of training examples: 373100


The neural network is trained using cross-entropy loss.

In [ ]:
def train_model(model, vocab, train_data_loader, optimizer, n_epochs, device):
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(n_epochs):
        start = time.time()
        n_batch = 0
        total_loss = 0
        model.train()      
        for train_x, train_y in tqdm(train_data_loader):
            optimizer.zero_grad() 
            train_x = train_x.to(device)
            train_y = train_y.to(device)
            logits = model(train_x)
            loss = loss_func(logits, train_y.long())
            loss.backward()
            optimizer.step()
            
            total_loss +=  loss.item()
            n_batch += 1
        
        print('Epoch:{:2d}/{}\t Loss: {:.4f} \t({:.2f}s)'.format(epoch + 1, n_epochs, total_loss / n_batch, time.time() - start))

Next the model and an <a href=https://jmlr.org/papers/volume12/duchi11a/duchi11a.pdf>Adagrad</a> optimizer are instantiated.

In [ ]:
# adjustable hyperparameters
BATCH_SIZE = 1024
LEARNING_RATE = 0.01
N_EPOCHS = 10
HIDDEN_SIZE = 300
DROPOUT_PROB = 0.1
EMBED_SIZE = 100
WEIGHT_DECAY = 1e-8
N_EMBEDDINGS = vocab.n_tokens # do not change
N_FEATURES = 48 # do not change
N_CLASSES = vocab.n_trans # do not change

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True, shuffle=True)
model = Model(N_EMBEDDINGS, EMBED_SIZE, N_FEATURES, HIDDEN_SIZE, N_CLASSES, DROPOUT_PROB).to(device)
optimizer = optim.Adagrad(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

The model is trained below.

In [ ]:
train_model(model, vocab, train_data_loader, optimizer, N_EPOCHS, device)

  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 1/10	 Loss: 0.7530 	(4.87s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 2/10	 Loss: 0.2635 	(3.68s)


  0%|          | 0/364 [00:00<?, ?it/s]

...

Epoch: 8/10	 Loss: 0.1543 	(3.70s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch: 9/10	 Loss: 0.1467 	(5.47s)


  0%|          | 0/364 [00:00<?, ?it/s]

Epoch:10/10	 Loss: 0.1407 	(3.54s)


## Evaluate Model

Now that the model is trained, it can be used to parse unseen sentences from a test set.

### Select Best Legal Prediction
The two functions combined below takes a trained model and a batch of parser configurations, and returns the highest probability <b>legal</b> predicted action for each parser configuration.

In [ ]:
def select_best_legal_action(parser_configs, predictions, n_labels):
    for p in range(predictions.shape[0]):
      parser_config = parser_configs[p]
      predictions[p, :] = predictions[p, :] + 100
      if len(parser_config.buffer) == 0: # illegal S action
        predictions[p, 2*n_labels] = 0
      if len(parser_config.stack) < 2: # illegal RA action
        predictions[p, n_labels:2*n_labels] = 0
      if len(parser_config.stack) < 3: # illegal LA action
        predictions[p, :n_labels] = 0

    preds = np.argmax(predictions, axis = 1)

    return preds

In [ ]:
def predict(model, vocab, parser_configs):
    model_device = next(model.parameters()).device
    
    x = np.array([extract_features(p) for p in parser_configs])
    x = torch.from_numpy(x).long().to(model_device)
    
    with torch.no_grad():
        pred = model(x)

    pred = pred.detach().cpu().numpy()
    pred = select_best_legal_action(parser_configs, pred, vocab.n_labels)
    actions = [vocab.id2tran[p] for p in pred]
    return actions

### Test Set Attachment Score

The following functions use the model to parse all sentences in the test set, and compute the attachment score. The <b>unlabeled attachment score</b> is the percentage of arcs in the test set for which the model gets the head correct. The <b>labeled attachment score</b> is the percentage of arcs for which the model gets <em>both</em> the head and the label correct. Thus, attachment score is a number between 0 and 100, and a higher score is better.

In [ ]:
def run_inference(sentences, model, vocab, batch_size=2000):
    N = len(sentences)

    # Initialize parser configs
    parser_configs = [None] * N
    for i in range(N):
        sent = sentences[i]
        parser_config = ParserConfiguration(sent, vocab)
        parser_configs[i] = parser_config

    parses_completed = [False] * N # Indicates whether a given parse is completed
    
    while sum(parses_completed) != N:

        # Get batch along with indices
        batch_idxes = []
        for idx in range(N):
            if not parses_completed[idx]: batch_idxes.append(idx)
            if len(batch_idxes) == batch_size: break
        batch = [parser_configs[idx] for idx in batch_idxes]

        # Make prediction, run a parse step, and check for completion
        transitions = predict(model, vocab, batch)
        for idx, parser, transition in zip(batch_idxes, batch, transitions):
            parser.parse_step(transition)
            if not parser.buffer.buffer and len(parser.stack.stack) == 1:
                parses_completed[idx] = True
    
    return [parser.dependencies for parser in parser_configs]

def transform_to_head_label(dependencies):
    head = [-1] * len(dependencies.arcs)
    label = [-1] * len(dependencies.arcs)
    for dep in dependencies.arcs:
        head[dep.dependent.idx-1] = dep.head.idx
        label[dep.dependent.idx-1] = dep.label_id  
    return head, label


def evaluate(model, vocab, dataset, eval_batch_size=5000):
    model.eval()
    sentences = [x[0] for x in dataset]
    gold_dependencies = [x[1] for x in dataset]
    pred_dependencies = run_inference(sentences, model, vocab, eval_batch_size)
    
    UAS, LAS = 0.0, 0.0

    all_tokens = 0
    
    for i in range(len(gold_dependencies)):
        assert len(gold_dependencies[i].arcs) == len(pred_dependencies[i].arcs)
        
        # Get gold answers
        gold_head, gold_label = transform_to_head_label(gold_dependencies[i])
        
        # Get predictions
        pred_head, pred_label = transform_to_head_label(pred_dependencies[i])
        
        assert len(gold_head) == len(pred_head) and len(gold_label) == len(pred_label)
        assert -1 not in gold_head + gold_label + pred_head + pred_label

        for pred_h, gold_h, pred_l, gold_l  in zip(pred_head, gold_head, pred_label, gold_label):
            UAS += (1 if pred_h == gold_h else 0)
            LAS += (1 if pred_h == gold_h and pred_l == gold_l else 0)
            all_tokens += 1
    return UAS / all_tokens * 100, LAS / all_tokens * 100, pred_dependencies

The attachment scores are computed below.

In [ ]:
UAS, LAS, test_predictions = evaluate(model, vocab, test_data)
print("Test Set Unlabeled Attachment Score:", UAS)
print("Test Set Labeled Attachment Score:", LAS)

Test Set Unlabeled Attachment Score: 83.10004771751233
Test Set Labeled Attachment Score: 80.4238905678384


## Qualitative Analysis

Here the model can be analyzed qualitatively to get a feel for the strengths and shortcomings of the model.

Some example sentences from the test set are printed. For each sentence, the gold (correct) dependency tree, the dependency tree predicted by the model, and a diagnostic of the gold tree are displayed. The diagnostic tree annotates the edges of the gold tree as follows:
* ✓: Edges for which the model predicted both the <b>correct head & label</b>
* ⍻: Edges for which the model predicted the <b>correct head but incorrect label</b>
* ×: Edges that are not in the tree (i.e., the model predicted the <b>incorrect head<b>).

In [ ]:
def diagnose(sentence, gold, pred):
    word, pos = [x.word for x in sentence], [x.pos for x in sentence]
    gold_head, gold_label = transform_to_head_label(gold)
    pred_head, pred_label = transform_to_head_label(pred)
    gold_label = [gold.vocab.id2tok[x][4:] for x in gold_label]
    pred_label = [gold.vocab.id2tok[x][4:] for x in pred_label]

    diff = ["✓"] * len(pred_head)
    for i in range(len(pred_head)):
        if gold_head[i]!=pred_head[i]: diff[i] = "×"
        elif gold_label[i]!=pred_label[i]: diff[i] = "⍻"
    unlabeled_score=sum([x in {"✓","⍻"}  for x in diff]) / len(diff)
    score=sum([x == "✓" for x in diff]) / len(diff)

    print("Your dependency tree:")
    sent = {"word": word[1:], "pos": pos[1:], "label": pred_label, "head": pred_head}
    display_sentence(sent)
    
    print("Gold dependency tree:")
    sent["label"], sent["head"] = gold_label, gold_head
    display_sentence(sent)
    
    print("Diagnostic of the gold tree:")
    sent["label"], sent["head"] = diff, gold_head
    display_sentence(sent)
    print('Unlabeled Attachment Score for this sentence:', unlabeled_score*100)
    print('Labeled Attachment Score for this sentence:', score*100, '\n')

def diagnose_sentences(idxes, data, preds, min_len, max_len, num_to_print=5):
    print('-'*100, '\n')
    for i in random.sample(list(filter(lambda x: len(data[x][0]) >= min_len and len(data[x][0]) <= max_len, idxes)), num_to_print):
        diagnose(data[i][0], data[i][1], preds[i])
        print('-'*100, '\n')

In [ ]:
MIN_SENT_LEN, MAX_SENT_LEN = 8, 17 # adjustable parameters
idxes=list(range(len(test_data))) # sample from all sentences
diagnose_sentences(idxes, test_data, test_predictions, MIN_SENT_LEN, MAX_SENT_LEN, num_to_print=5)

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 80.0
Labeled Attachment Score for this sentence: 80.0 

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


...

Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 85.71428571428571
Labeled Attachment Score for this sentence: 85.71428571428571 

---------------------------------------------------------------------------------------------------- 

Your dependency tree:


Gold dependency tree:


Diagnostic of the gold tree:


Unlabeled Attachment Score for this sentence: 100.0
Labeled Attachment Score for this sentence: 100.0 

---------------------------------------------------------------------------------------------------- 

